In [1]:
#pip install cmake gym[atari] scipy

In [3]:
import gym

In [4]:
# Creation of the environment
env = gym.make("Taxi-v3", render_mode="human")

#env = gym.make("Taxi-v3").env
#env.render()

In [5]:
env.reset() # reset the environment to a new random state
env.render() 

print("Action Space {}". format(env.action_space))
print("State Space {}.". format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500).


In [6]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()


State: 328


In [7]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [8]:
env.close()

In [9]:

env = gym.make("Taxi-v3", render_mode="ansi")
env.reset()

# without reinforcement learning
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, truncate, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 2965
Penalties incurred: 935


C:\Users\masanz\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [9]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        #print(frame['frame'].getvalue())
        print(f"{frame['frame']}")
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)

Timestep: 768
State: 407
Action: 0
Reward: -1


KeyboardInterrupt: 

In [10]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [11]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()
    state= state[0]

    epochs, penalties, reward, = 0, 0, 0
    done = False


    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values
        

        next_state, reward, done, truncate, info = env.step(action) 
  
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
  
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        #print(state[0], action, new_value)
        q_table[state,action] = new_value
        #print(state[0], action, new_value)
        #print(reward)
        
        if reward == -10:
            penalties += 1

        state= next_state
        epochs += 1
        #print(state)
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: total: 31 s
Wall time: 1min 1s


In [12]:
q_table

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ -2.41837066,  -2.3639511 ,  -2.41837065,  -2.36395109,
         -2.27325184, -11.36395096],
       [ -1.87014393,  -1.45024   ,  -1.87014399,  -1.45024003,
         -0.7504    , -10.45023968],
       ...,
       [ -1.00487504,   0.41599993,  -1.0160436 ,  -1.17402336,
         -4.6844618 ,  -4.64021351],
       [ -2.15870569,  -2.12207627,  -2.15802819,  -2.12207598,
         -7.58135084,  -4.46154102],
       [  3.56818763,   0.95261556,   3.5455277 ,  11.        ,
         -3.03910601,  -2.67125441]])

In [13]:
#env.close()

In [14]:
#env = gym.make("Taxi-v3", render_mode="ansi")
#env.reset()

In [15]:

"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

framesql = [] # for animation

for _ in range(episodes):
    state = env.reset()
    state= state[0]
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, truncate, info = env.step(action)

        if reward == -10:
            penalties += 1
        
        epochs += 1

    total_penalties += penalties
    total_epochs += epochs
    # Put each rendered frame into dict for animation
    framesql.append({
    'frame': env.render(),
    'state': state,
    'action': action,
    'reward': reward,
    'epoch': epochs   
        }
    )

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")



Results after 100 episodes:
Average timesteps per episode: 13.55
Average penalties per episode: 0.0


In [19]:
env.close()
env = gym.make("Taxi-v3", render_mode="human")


In [20]:
# After 10 000 episodes, our Q-table can be used as a "cheatsheet" to play FrozenLake"
# Here the agent plays FrozenLake

env.reset()

max_steps = 100

for episode in range(10):
    state = env.reset()
    state=state[0]
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(q_table[state,:])
        
        new_state, reward, done, truncate, info = env.step(action)

        env.render()
        
        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            
            # We print the number of step it took.
            print("Number of steps", step)
            break
        state = new_state
env.close()

****************************************************
EPISODE  0
Number of steps 11
****************************************************
EPISODE  1
Number of steps 16
****************************************************
EPISODE  2
Number of steps 12
****************************************************
EPISODE  3
Number of steps 15
****************************************************
EPISODE  4
Number of steps 8
****************************************************
EPISODE  5
Number of steps 10
****************************************************
EPISODE  6
Number of steps 12
****************************************************
EPISODE  7
Number of steps 12
****************************************************
EPISODE  8
Number of steps 9
****************************************************
EPISODE  9
Number of steps 11
